In [1]:
%pip install python-dotenv --upgrade --quiet faiss-cpu langchain langchain-google-genai langchain-huggingface sentence-transformers langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
import numpy as np

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Setup complete.")

Enter your Google API Key: ··········


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setup complete.


In [3]:
def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    a, b = np.array(a), np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [4]:
words = ["King", "Queen", "Man", "Woman", "Paris", "France", "Car", "Automobile"]

vectors = {word: embeddings.embed_query(word) for word in words}

print(f"Vector dimensionality: {len(list(vectors.values())[0])}\n")

pairs = [
    ("King",   "Queen"),
    ("King",   "Man"),
    ("Paris",  "France"),
    ("Car",    "Automobile"),
    ("King",   "Car"),
    ("Woman",  "Car"),
]

print(f"{'Pair':<30} {'Similarity':>10}")
print("-" * 42)
for w1, w2 in pairs:
    sim = cosine_similarity(vectors[w1], vectors[w2])
    print(f"{w1 + ' vs ' + w2:<30} {sim:>10.4f}")

Vector dimensionality: 384

Pair                           Similarity
------------------------------------------
King vs Queen                      0.6807
King vs Man                        0.3216
Paris vs France                    0.7331
Car vs Automobile                  0.8645
King vs Car                        0.2876
Woman vs Car                       0.4002


In [5]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

docs = [
    Document(page_content="NovaCorp was founded in 2021 by Aisha Patel and Marcus Lee."),
    Document(page_content="NovaCorp's flagship product is an AI-powered irrigation system called AquaMind."),
    Document(page_content="AquaMind uses soil moisture sensors and weather forecasts to reduce water usage by up to 40%."),
    Document(page_content="NovaCorp is headquartered in Bangalore, India, with a satellite office in Amsterdam."),
    Document(page_content="The CEO of NovaCorp is Aisha Patel. The CTO is Marcus Lee."),
    Document(page_content="NovaCorp raised a Series A of $12 million in March 2023 led by GreenVentures."),
]

print(f"Knowledge base has {len(docs)} documents.")

Knowledge base has 6 documents.


In [6]:
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

print("FAISS index built successfully.")

FAISS index built successfully.


In [7]:
template = """
You are a helpful assistant. Answer the question based ONLY on the context below.
If the answer is not in the context, say "I don't have that information."

Context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
questions = [
    "Who founded NovaCorp?",
    "What does AquaMind do?",
    "How much funding did NovaCorp raise and who led it?",
    "Where is NovaCorp headquartered?",
    "What is NovaCorp's stock price?"
]

for q in questions:
    print(f"Q: {q}")
    print(f"A: {chain.invoke(q)}")
    print()

Q: Who founded NovaCorp?
A: Aisha Patel and Marcus Lee founded NovaCorp.

Q: What does AquaMind do?
A: AquaMind is an AI-powered irrigation system that uses soil moisture sensors and weather forecasts to reduce water usage by up to 40%.

Q: How much funding did NovaCorp raise and who led it?
A: NovaCorp raised $12 million led by GreenVentures.

Q: Where is NovaCorp headquartered?
A: NovaCorp is headquartered in Bangalore, India.

Q: What is NovaCorp's stock price?
A: I don't have that information.



In [10]:
query = "Who is the CTO of NovaCorp?"
retrieved_docs = retriever.invoke(query)

print(f"Query: '{query}'")
print(f"Retrieved {len(retrieved_docs)} document(s):\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"  Doc {i}: {doc.page_content}")

Query: 'Who is the CTO of NovaCorp?'
Retrieved 2 document(s):

  Doc 1: The CEO of NovaCorp is Aisha Patel. The CTO is Marcus Lee.
  Doc 2: NovaCorp was founded in 2021 by Aisha Patel and Marcus Lee.


In [11]:
import faiss
import time

d  = 128
nb = 100_000
nq = 100
k  = 5

np.random.seed(42)
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')
print(f"Dataset: {nb:,} vectors | Dimensions: {d} | Queries: {nq}")

Dataset: 100,000 vectors | Dimensions: 128 | Queries: 100


In [12]:
def benchmark(index, xq, k=5, label=""):
    """Time how long it takes to run nq queries against the index."""
    start = time.perf_counter()
    D, I = index.search(xq, k)
    elapsed = (time.perf_counter() - start) * 1000  # ms
    print(f"{label:<20} | {elapsed:>8.2f} ms total | {elapsed/len(xq):>6.3f} ms/query")
    return I

print(f"{'Index':<20} | {'Total Time':>14} | {'Per Query':>14}")
print("-" * 50)

Index                |     Total Time |      Per Query
--------------------------------------------------


In [13]:
flat = faiss.IndexFlatL2(d)
flat.add(xb)
I_flat = benchmark(flat, xq, k, label="Flat (Brute Force)")

Flat (Brute Force)   |    49.10 ms total |  0.491 ms/query


In [14]:
nlist = 200
quantizer = faiss.IndexFlatL2(d)
ivf = faiss.IndexIVFFlat(quantizer, d, nlist)
ivf.train(xb)
ivf.add(xb)
ivf.nprobe = 10
I_ivf = benchmark(ivf, xq, k, label="IVF (nprobe=10)")

IVF (nprobe=10)      |    18.45 ms total |  0.184 ms/query


In [15]:
M = 16
hnsw = faiss.IndexHNSWFlat(d, M)
hnsw.add(xb)
I_hnsw = benchmark(hnsw, xq, k, label="HNSW (M=16)")

HNSW (M=16)          |     7.64 ms total |  0.076 ms/query


In [16]:
m = 8
pq = faiss.IndexPQ(d, m, 8)
pq.train(xb)
pq.add(xb)
I_pq = benchmark(pq, xq, k, label="PQ (m=8)")

PQ (m=8)             |    24.63 ms total |  0.246 ms/query


In [17]:
def recall_at_k(I_approx, I_true, k=5):
    """Fraction of true top-k results found by approximate index."""
    hits = sum(
        len(set(I_approx[i, :k]) & set(I_true[i, :k]))
        for i in range(len(I_true))
    )
    return hits / (len(I_true) * k)

print("\nRecall@5 vs Flat (Ground Truth):")
print(f"  IVF  : {recall_at_k(I_ivf,  I_flat):.2%}")
print(f"  HNSW : {recall_at_k(I_hnsw, I_flat):.2%}")
print(f"  PQ   : {recall_at_k(I_pq,   I_flat):.2%}")


Recall@5 vs Flat (Ground Truth):
  IVF  : 21.00%
  HNSW : 12.20%
  PQ   : 4.40%
